In [214]:
import folium
import numpy as np 
import pandas as pd
from datetime import datetime
import json
from shapely.geometry import shape, Point
import geopandas as gpd

from Clean_Data import remove_outliers
from Clean_Data import remove_missing
from Load_Data import load_data

In [215]:
start_time = datetime.fromisoformat("2018-05-05T00:00:00")
df = load_data(48.7, 48.85, 9.05, 9.3, start_time, 1)

In [216]:
df = remove_missing(df)
df = remove_outliers(df)

6439 observations with missing values were removed from the data frame
32 outlier observations were deleted


In [217]:
df = df.loc[range(200)]

In [6]:
def map_data(df):
    '''
    Maps the PM10 and PM2.5 concentration 
    
    INPUTS:
    df:        A pandas dataframe containing PM10 and Pm2.5 measurements, the location and time of the measurement
    
    OUTPUTS:
    index:     An HTML file with an interactive map
    '''
    
    # define the covered area of the map
    min_lat, max_lat, min_lon, max_lon = min(df[['lat','lon']]) - 0.1, max(df[['lat','lon']]) + 0.1
    
    
    

In [218]:
plz = gpd.read_file("mapping_data/plz_stuggi.geojson")

geo_df = gpd.GeoDataFrame(df,geometry = gpd.points_from_xy(df.lon, df.lat,), crs = 4326)

plz_pm = gpd.sjoin(plz_dat,geo_df)
plz_pm['polygon_id'] = plz_pm.index
plz_pm = plz_pm.reset_index()
plz_geom = plz_pm[['polygon_id','geometry']]
plz_pm = plz_pm[['polygon_id',"measurement_PM10",'measurement_PM2.5','time']]
plz_pm['time'] = pd.to_datetime(plz_pm['time'])


#plz_pm = plz_pm.reset_index(drop = True)
#plz_pm['geometry_point'] = geo_df['geometry'][plz_pm["index_right"]].to_list() 
#plz_pm_grouped = group_by()
#plz_pm = plz_pm[['geometry',"measurement_PM10",'measurement_PM2.5','time']]

In [219]:
gr_plz_pm = plz_pm.groupby([pd.Grouper(key = "time",freq='1H'), 'polygon_id']).mean()
gr_plz_pm = gr_plz_pm.reset_index()
#gr_plz_pm['geometry'] = plz['geometry'][gr_plz_pm['polygon_id']]

In [220]:
m = folium.Map(location=center, zoom_start=12)
center = [np.median(df['lat']), np.median(df['lon'])]

folium.Choropleth(
    geo_data = plz_geom,
    name = "choropleth",
    data = gr_plz_pm,
    columns = ['polygon_id','measurement_PM10'],
    key_on = 'feature.properties.polygon_id',
    
).add_to(m)
#for jj in range(df.shape[0]):
#    folium.Marker(location = [df['lat'][jj],df['lon'][jj]]).add_to(m)

m

In [184]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"
state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
state_data = pd.read_csv(state_unemployment)

plz = gpd.read_file(state_geo)